<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_fc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition with LSTM 
Acoustic features are loaded directly from data directory

In [1]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
!git clone https://github.com/bagustris/ravdess_song.git

Cloning into 'ravdess_song'...
remote: Enumerating objects: 1116, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1116 (delta 28), reused 26 (delta 10), pack-reused 1057
Receiving objects: 100% (1116/1116), 232.30 MiB | 24.49 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Checking out files: 100% (1030/1030), done.


In [3]:
cd ravdess_song/

/content/ravdess_song


In [5]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.load('data/x.npy', allow_pickle=True)
y = np.load('data/y.npy', allow_pickle=True)

In [ ]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [6]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [10]:
# functio to build LSTM model
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [11]:
# create the model
model = model_lstm()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 386)               1544      
_________________________________________________________________
dense (Dense)                (None, 256)               99072     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 233,742
Trainable params: 232,970
Non-trainable params: 772
______________________________________________

In [12]:
# train the Dense model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 1s 4ms/step - loss: 1.2509 - accuracy: 0.4989
Epoch 2/100
57/57 [==============================] - 0s 4ms/step - loss: 0.5902 - accuracy: 0.7593
Epoch 3/100
57/57 [==============================] - 0s 4ms/step - loss: 0.4794 - accuracy: 0.8165
Epoch 4/100
57/57 [==============================] - 0s 4ms/step - loss: 0.3518 - accuracy: 0.8516
Epoch 5/100
57/57 [==============================] - 0s 4ms/step - loss: 0.2965 - accuracy: 0.8846
Epoch 6/100
57/57 [==============================] - 0s 4ms/step - loss: 0.2587 - accuracy: 0.9022
Epoch 7/100
57/57 [==============================] - 0s 4ms/step - loss: 0.2737 - accuracy: 0.8846
Epoch 8/100
57/57 [==============================] - 0s 3ms/step - loss: 0.2418 - accuracy: 0.9143
Epoch 9/100
57/57 [==============================] - 0s 4ms/step - loss: 0.2370 - accuracy: 0.9176
Epoch 10/100
57/57 [==============================] - 0s 4ms/step - loss: 0.2620 - accuracy: 0.8978
Epoch 11/

In [13]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 0s 2ms/step - loss: 0.8785 - accuracy: 0.8431
Loss:  0.8784763216972351 --> Accuracy:  0.843137264251709
